In [2]:
import pandas as pd
import numpy as np

In [3]:
%pwd

'/Users/yvesgreatti/github/kaggle_favorita_grocery_sales_forecasting/notebook'

In [4]:
dtype_dict={"id":np.uint32,
            "store_nbr":np.uint8,
            "item_nbr":np.uint32,
            "unit_sales":np.float32
           }
df = pd.read_csv("../data/train.csv",dtype=dtype_dict)


/var/folders/w3/d06m5l_97yn4bnxd5t6cgbjr0000gn/T/ipykernel_90234/2401172864.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/train.csv",dtype=dtype_dict)


In [5]:
len(df)

125497040

In [6]:
df["date"] = pd.to_datetime(df["date"])
df["unit_sales"] = df["unit_sales"].fillna(0)
df['unit_sales'] = df['unit_sales'].clip(lower=0)
df.rename(columns={"item_nbr":"item","store_nbr":"store"}, inplace=True)
df["store_item"] = df["store"].astype(str).str.cat(df["item"].astype(str), sep="_")

# assume df["date"] is datetime64[ns]
min_date = df["date"].min()

# compute the Monday of the week containing min_date
week_start = min_date - pd.Timedelta(days=min_date.weekday())
week_start
#if min_date is already a Monday, keep that; otherwise bump to next week’s Monday
if min_date.weekday() == 0:
    first_monday = week_start
else:
    first_monday = week_start + pd.Timedelta(days=7)

print("min_date:", min_date)
print("first day of the first full week:", first_monday)
df = df[df['date'] >= first_monday]
cols = ['date', 'store_item', 'store', 'item'] + [c for c in df.columns 
                                  if c not in ('date', 'store_item', 'store_nbr', 'item_nbr')]
df = df[cols]
df = df.sort_values(by=['date', 'store_item'])
df = df.reset_index(drop=True)
len(df)

min_date: 2013-01-01 00:00:00
first day of the first full week: 2013-01-07 00:00:00


125290231

In [7]:
df.head()

,date,store_item,store,item,id,store,item,unit_sales,onpromotion
0,2013-01-07,10_1003679,10,1003679,216580,10,1003679,3.000,NaN
1,2013-01-07,10_1009512,10,1009512,216581,10,1009512,1.000,NaN
2,2013-01-07,10_1012473,10,1012473,216582,10,1012473,25.000,NaN
3,2013-01-07,10_1012788,10,1012788,216583,10,1012788,2.000,NaN
4,2013-01-07,10_1017349,10,1017349,216584,10,1017349,8.494,NaN


In [8]:
df["date"].unique()

<DatetimeArray>
['2013-01-07 00:00:00', '2013-01-08 00:00:00', '2013-01-09 00:00:00',
 '2013-01-10 00:00:00', '2013-01-11 00:00:00', '2013-01-12 00:00:00',
 '2013-01-13 00:00:00', '2013-01-14 00:00:00', '2013-01-15 00:00:00',
 '2013-01-16 00:00:00',
 ...
 '2017-08-06 00:00:00', '2017-08-07 00:00:00', '2017-08-08 00:00:00',
 '2017-08-09 00:00:00', '2017-08-10 00:00:00', '2017-08-11 00:00:00',
 '2017-08-12 00:00:00', '2017-08-13 00:00:00', '2017-08-14 00:00:00',
 '2017-08-15 00:00:00']
Length: 1678, dtype: datetime64[ns]

In [11]:
print("Unique (store, item) pairs:", df["store_item"].nunique())

Unique (store, item) pairs: 174678


In [9]:
 # Save to a new CSV
df.to_csv(
    "../output/data/train_preprocessed.csv",
    index=False
)

# Preprocessing Test Data

In [4]:
dtype_dict={"id":np.uint32,
            "store_nbr":np.uint8,
            "item_nbr":np.uint32,
            "onpromotion":np.float32
           }
test_df = pd.read_csv("../data/test.csv", dtype=dtype_dict)
test_df.head()

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,0.0
1,125497041,2017-08-16,1,99197,0.0
2,125497042,2017-08-16,1,103501,0.0
3,125497043,2017-08-16,1,103520,0.0
4,125497044,2017-08-16,1,103665,0.0


In [5]:
#test_df.drop("id", axis=1, inplace=True)
test_df["date"] = pd.to_datetime(test_df["date"])
test_df["unit_sales"] = 0
test_df.rename(columns={"item_nbr":"item","store_nbr":"store"}, inplace=True)
test_df["store_item"] = test_df["store"].astype(str).str.cat(test_df["item"].astype(str), sep="_")
len(test_df)

3370464

In [6]:
cols = ['id','date', 'store_item', 'store', 'item'] + [c for c in test_df.columns 
                                  if c not in ('id','date', 'store_item', 'store', 'item')]
test_df = test_df[cols]
test_df = test_df.sort_values(by=['date', 'store_item'])
test_df = test_df.reset_index(drop=True)
test_df.head(5)

,id,date,store_item,store,item,onpromotion,unit_sales
0,125533483,2017-08-16,10_1000866,10,1000866,0.0,0
1,125533484,2017-08-16,10_1001305,10,1001305,0.0,0
2,125533485,2017-08-16,10_1003679,10,1003679,0.0,0
3,125533486,2017-08-16,10_1004545,10,1004545,0.0,0
4,125533487,2017-08-16,10_1004550,10,1004550,1.0,0


In [7]:
print("Unique (store, item) pairs:", test_df["store_item"].nunique())

Unique (store, item) pairs: 210654


In [8]:
test_df.to_csv(
    "../output/data/20250530_test_preprocessed.csv",
    index=False
)

In [17]:
item_df = pd.read_csv("../data/items.csv")
item_df["item"] = item_df["item_nbr"]
item_df.head()


,item_nbr,family,class,perishable,item
0,96995,GROCERY I,1093,0,96995
1,99197,GROCERY I,1067,0,99197
2,103501,CLEANING,3008,0,103501
3,103520,GROCERY I,1028,0,103520
4,103665,BREAD/BAKERY,2712,1,103665


In [18]:
w_perish = (
    item_df
      .query("perishable == 1")[["item"]]
      .assign(weight=1.25)
)

# build the “non‑perishable” weights:
w_non    = (
    item_df
      .query("perishable == 0")[["item"]]
      .assign(weight=1.00)
)

# concatenate and reset index:
weights_df = pd.concat([w_perish, w_non], ignore_index=True)

# if you want to preserve original order:
weights_df = (
    item_df[['item']]
      .merge(weights_df, on='item')
)

weights_df.head()

,item,weight
0,96995,1.00
1,99197,1.00
2,103501,1.00
3,103520,1.00
4,103665,1.25


In [19]:
weights_df.to_csv(
    "../output/data/item_weights.csv",
    index=False
)